<a href="https://colab.research.google.com/github/pawansharmaaaa/Wav2Lip_Mod/blob/master/GFPGAN_Wav2Lip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prerequisites

Initialization

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!mkdir /content/Data

In [3]:
!cp -ri "/content/drive/MyDrive/W2L_GFPMod/TAudio.wav" "/content/drive/MyDrive/W2L_GFPMod/unfiltered_input.mp4" /content/Data

Wav2Lip

In [4]:
!git clone https://github.com/pawansharmaaaa/Wav2Lip_Mod.git

Cloning into 'Wav2Lip_Mod'...
remote: Enumerating objects: 407, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 407 (delta 23), reused 1 (delta 0), pack-reused 360
Receiving objects: 100% (407/407), 560.25 KiB | 16.48 MiB/s, done.
Resolving deltas: 100% (221/221), done.


In [5]:
!cp -ri "/content/drive/MyDrive/W2L_GFPMod/wav2lip.pth" "/content/drive/MyDrive/W2L_GFPMod/wav2lip_gan.pth" /content/Wav2Lip_Mod/checkpoints

In [6]:
!pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip_Mod && pip install -r requirements.txt

Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.12.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? Y
  Successfully uninstalled tensorflow-2.12.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.1/213.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 22.4 MB/s 

**DEPRECATED**

In [ ]:
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "Wav2Lip_Mod/face_detection/detection/sfd/s3fd.pth"

GFPGAN

In [ ]:
!cd Wav2Lip_Mod && git clone https://github.com/TencentARC/GFPGAN.git

In [ ]:
!pip install basicsr facexlib

In [ ]:
!cd Wav2Lip_Mod/GFPGAN && pip install -r requirements.txt

In [ ]:
!cd Wav2Lip_Mod/GFPGAN && python setup.py develop

In [ ]:
!pip install realesrgan

In [ ]:
!cd Wav2Lip_Mod/GFPGAN && wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models

In [ ]:
!cd Wav2Lip_Mod/GFPGAN && python inference_gfpgan.py -i /content/GFPGAN/inputs/test2.jpg -o results/new -v 1.3 -s 2

In [ ]:
!pip install gfpgan

In [ ]:
!cp -ri "/content/Data/w2l_input_video.mp4" /content/drive/MyDrive/Finalized_notebook

#**No Face Filter** *(Remove Frames Which do not contain faces from the Video)*

In [10]:
import cv2
import face_recognition
import gc
import tqdm

# Function to detect face in a frame
def detect_face(frames):
    batch_face_locations = face_recognition.batch_face_locations(frames, number_of_times_to_upsample=0)
    return batch_face_locations

# Open the video file for reading
video_path = '/content/Data/unfiltered_input.mp4'
video_capture = cv2.VideoCapture(video_path)

# Get video properties
fps = video_capture.get(cv2.CAP_PROP_FPS)
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create video writers for face.mp4
face_writer = cv2.VideoWriter('/content/Data/filtered_input_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

batch_size = 16
frames = []
while True:
    ret, frame = video_capture.read()

    if not ret:
        break

    # Detect faces in the frame
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames.append(frame)

    if len(frames) == batch_size:
      frames_mem = frames.copy()
      batch_faces = detect_face(frames_mem)

      for frame_no, face_locations in enumerate(tqdm.tqdm(batch_faces, total=batch_size)):
        if len(face_locations) == 1:
            # Write the frame to input_video.mp4 if a face is detected
            pos_frame = cv2.cvtColor(frames_mem[frame_no], cv2.COLOR_RGB2BGR)
            face_writer.write(pos_frame)
      gc.collect()
      frames = []

# Release video capture, writers and clear garbage
video_capture.release()
face_writer.release()
cv2.destroyAllWindows()

100%|██████████| 16/16 [00:00<00:00, 94.15it/s]


#Running Wav2Lip

In [ ]:
!cd Wav2Lip_Mod && python inf_w2l.py --checkpoint_path checkpoints/wav2lip.pth --face "/content/Data/filtered_input_video.mp4" --audio "/content/Data/TAudio.wav" --resize_factor 2 --upscale

Streaming output truncated to the last 5000 lines.
	Tile 7/15
	Tile 8/15
	Tile 9/15
	Tile 10/15
	Tile 11/15
	Tile 12/15
	Tile 13/15
	Tile 14/15
	Tile 15/15
	Tile 1/15
	Tile 2/15
	Tile 3/15
	Tile 4/15
	Tile 5/15
	Tile 6/15
	Tile 7/15
	Tile 8/15
	Tile 9/15
	Tile 10/15
	Tile 11/15
	Tile 12/15
	Tile 13/15
	Tile 14/15
	Tile 15/15
	Tile 1/15
	Tile 2/15
	Tile 3/15
	Tile 4/15
	Tile 5/15
	Tile 6/15
	Tile 7/15
	Tile 8/15
	Tile 9/15
	Tile 10/15
	Tile 11/15
	Tile 12/15
	Tile 13/15
	Tile 14/15
	Tile 15/15
	Tile 1/15
	Tile 2/15
	Tile 3/15
	Tile 4/15
	Tile 5/15
	Tile 6/15
	Tile 7/15
	Tile 8/15
	Tile 9/15
	Tile 10/15
	Tile 11/15
	Tile 12/15
	Tile 13/15
	Tile 14/15
	Tile 15/15
	Tile 1/15
	Tile 2/15
	Tile 3/15
	Tile 4/15
	Tile 5/15
	Tile 6/15
	Tile 7/15
	Tile 8/15
	Tile 9/15
	Tile 10/15
	Tile 11/15
	Tile 12/15
	Tile 13/15
	Tile 14/15
	Tile 15/15
	Tile 1/15
	Tile 2/15
	Tile 3/15
	Tile 4/15
	Tile 5/15
	Tile 6/15
	Tile 7/15
	Tile 8/15
	Tile 9/15
	Tile 10/15
	Tile 11/15
	Tile 12/15
	Tile 13/15
	Tile 14/15
	